### Limpeza dos csv brutos do Siconfi

In [9]:
import pandas as pd
import os

# Definir os caminhos (relativos ao local do notebook)
input_dir = '../data/raw/siconfi/raw'
output_dir = '../data/raw/siconfi/clean'

# Criar a pasta de destino caso ela não exista
os.makedirs(output_dir, exist_ok=True)

# Listar todos os arquivos CSV na pasta raw
files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

print(f"Iniciando processamento de {len(files)} arquivos...")

for file_name in files:
    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)
    
    try:
        # 1. Lemos pulando as 3 linhas e definindo o encoding original
        # sep=';' porque é o padrão de CSVs brasileiros/Siconfi
        df = pd.read_csv(input_path, skiprows=3, sep=';', encoding='latin1')
        
        # 2. Salvamos no destino com encoding utf-8 e sem o índice do pandas
        df.to_csv(output_path, index=False, encoding='utf-8', sep=';')
        
        print(f"✅ Sucesso: {file_name}")
        
    except Exception as e:
        print(f"❌ Erro ao processar {file_name}: {e}")

print("\nProcessamento concluído!")

Iniciando processamento de 13 arquivos...
✅ Sucesso: finbra_mun_despesas-por-funcao_2013.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2014.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2015.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2016.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2017.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2018.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2019.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2020.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2021.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2022.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2023.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2024.csv
✅ Sucesso: finbra_mun_despesas-por-funcao_2025.csv

Processamento concluído!


### Selecionar apenas as contas

06.182 - Defesa Civil (?) - repetido

15.451 - Infraestrutura Urbana (?) - Saneamento e Gestão ambiental é mais especifico

17 - Saneamento

18 - Gestão Ambiental

25.751 - Conservação de Energia (?)

25.754 - Biocombustíveis (?)

In [10]:
import pandas as pd
import os

# Caminhos das pastas
input_dir = '../data/raw/siconfi/clean'
output_dir = '../data/raw/siconfi/clean_filter'

# Lista de valores para filtrar na coluna 'Conta'
contas_alvo = [
    "17 - Saneamento",
    "18 - Gestão Ambiental"
]

# Criar a pasta filter se não existir
os.makedirs(output_dir, exist_ok=True)

# Listar arquivos na pasta siconfi (apenas CSVs)
files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

print(f"Iniciando a filtragem de {len(files)} arquivos...")

for file_name in files:
    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)
    
    try:
        # Lendo o arquivo já limpo (UTF-8)
        df = pd.read_csv(input_path, sep=';', encoding='utf-8')
        
        # Aplicando o filtro na coluna 'Conta'
        df_filtrado = df[df['Conta'].isin(contas_alvo)]
        
        # Salvando o resultado
        df_filtrado.to_csv(output_path, index=False, sep=';', encoding='utf-8')
        
        print(f"✔️ Filtrado: {file_name} ({len(df_filtrado)} linhas encontradas)")
        
    except Exception as e:
        print(f"❌ Erro ao filtrar {file_name}: {e}")

print("\nFiltragem concluída com sucesso!")

Iniciando a filtragem de 13 arquivos...
✔️ Filtrado: finbra_mun_despesas-por-funcao_2013.csv (25164 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2014.csv (25059 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2015.csv (26411 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2016.csv (24796 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2017.csv (26410 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2018.csv (27600 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2019.csv (27820 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2020.csv (26924 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2021.csv (27948 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2022.csv (29326 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2023.csv (30095 linhas encontradas)
✔️ Filtrado: finbra_mun_despesas-por-funcao_2024.csv (29080 linhas encontradas)


### Padronização e calculo do valor per capita
- Remover as colunas 'Instituição' e 'Identificador da Conta'
- Alterar o nome da coluna 'Cod.IBGE' para 'cd_mun'
- Padronizar o nome de todas as colunas, passando elas para minúsculo, escreva 'População' como 'populacao' e coloque underlines em espaçoes
- Calcular o valor per capita (utilizando a coluna 'Valor' e 'População') e salve na coluna 'valor_per_capita'
- Selecionar apenas as 'Despesas Liquidadas'

In [13]:
import pandas as pd
import os

input_dir = '../data/raw/siconfi/clean_filter'
output_dir = '../data/raw/siconfi/clean_filter_per-capita'
os.makedirs(output_dir, exist_ok=True)

def padronizar_colunas(col):
    col = str(col).lower()
    # Remove acentos e padroniza termos
    col = col.replace('ç', 'c').replace('ã', 'a').replace('é', 'e').replace('õ', 'o')
    col = col.replace(' ', '_')
    return col

files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

for file_name in files:
    path = os.path.join(input_dir, file_name)
    try:
        # IMPORTANTE: decimal=',' faz o pandas entender que 10,50 é 10.5
        df = pd.read_csv(path, sep=';', encoding='utf-8', decimal=',')
        
        # 1. Padronizar nomes de colunas
        df.columns = [padronizar_colunas(c) for c in df.columns]
        
        # 2. Filtrar apenas Despesas Liquidadas
        if 'coluna' in df.columns:
            df = df[df['coluna'] == 'Despesas Liquidadas']
        
        # 3. Renomear e remover colunas
        df = df.rename(columns={'cod.ibge': 'cd_mun'})
        cols_para_remover = ['instituicao', 'identificador_da_conta']
        df = df.drop(columns=[c for c in cols_para_remover if c in df.columns], errors='ignore')

        # 4. Garantir que valor e populacao são numéricos e tratar nulos
        # O decimal=',' no read_csv já resolve 90% dos casos, o to_numeric limpa o resto
        for col in ['valor', 'populacao']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
        
        if 'valor' in df.columns and 'populacao' in df.columns:
            # 5. Cálculo do Per Capita
            # Usando uma lógica simples: onde populacao > 0, divide. Senão, 0.
            df['valor_per_capita'] = 0.0
            mask = df['populacao'] > 0
            df.loc[mask, 'valor_per_capita'] = df.loc[mask, 'valor'] / df.loc[mask, 'populacao']
            
            # Salvar
            output_path = os.path.join(output_dir, file_name)
            df.to_csv(output_path, index=False, sep=';', encoding='utf-8')
            print(f"✅ Processado: {file_name} | Linhas: {len(df)}")
        else:
            print(f"⚠️ Colunas não encontradas em {file_name}: {df.columns.tolist()}")

    except Exception as e:
        print(f"❌ Erro em {file_name}: {e}")

print("\n--- Processo finalizado! Verifique os valores agora. ---")

✅ Processado: finbra_mun_despesas-por-funcao_2013.csv | Linhas: 6567
✅ Processado: finbra_mun_despesas-por-funcao_2014.csv | Linhas: 6434
✅ Processado: finbra_mun_despesas-por-funcao_2015.csv | Linhas: 6717
✅ Processado: finbra_mun_despesas-por-funcao_2016.csv | Linhas: 6661
✅ Processado: finbra_mun_despesas-por-funcao_2017.csv | Linhas: 6791
✅ Processado: finbra_mun_despesas-por-funcao_2018.csv | Linhas: 6995
✅ Processado: finbra_mun_despesas-por-funcao_2019.csv | Linhas: 7134
✅ Processado: finbra_mun_despesas-por-funcao_2020.csv | Linhas: 7277
✅ Processado: finbra_mun_despesas-por-funcao_2021.csv | Linhas: 7287
✅ Processado: finbra_mun_despesas-por-funcao_2022.csv | Linhas: 7607
✅ Processado: finbra_mun_despesas-por-funcao_2023.csv | Linhas: 7561
✅ Processado: finbra_mun_despesas-por-funcao_2024.csv | Linhas: 7577
✅ Processado: finbra_mun_despesas-por-funcao_2025.csv | Linhas: 419

--- Processo finalizado! Verifique os valores agora. ---
